### Manipulation of PEAKS de novo results of _T. weiss_  LC-MS/MS data using Python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >80% ALC
    from Thermo Fusion tribrid runs at the UW Proteomics Resource center, January 2021
    combined from multiple injections

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
cd /home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_332-T12nd-all_DENOVO_100/

/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_332-T12nd-all_DENOVO_100


In [4]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks332 = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_332-T12nd-all_DENOVO_100/332-T12nd-all_DENOVO_100_DN.csv")

print("# redundant Peaks peptides >80% ALC in combined dataframe:", len(peaks332))

# look at the dataframe
peaks332.head()

# redundant Peaks peptides >80% ALC in combined dataframe: 2713


,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,Area,Mass,ppm,PTM,local confidence (%),tag (>=0%),mode
0,22,14158,20210115_Weissrot_332_T12_nodigest_DDA_120min_...,HEDWQLK,7,98,7,478.2356,2,48.27,2130000.0,954.4559,0.7,NaN,98 100 99 98 95 100 99,HEDWQLK,HCD
1,22,25708,20210115_Weissrot_332_T12_nodigest_DDA_120min_...,LQNLHLL,7,98,7,425.7617,2,85.28,411000.0,849.5072,2.0,NaN,96 97 98 100 100 100 100,LQNLHLL,HCD
2,22,6806,20210115_Weissrot_332_T12_nodigest_DDA_120min_...,KR(+15.99)LVVEN,7,98,7,437.2560,2,25.49,979000.0,872.5079,-12.1,Oxidation(KPRY),98 99 100 99 95 99 99,KR(+15.99)LVVEN,HCD
3,22,26403,20210115_Weissrot_332_T12_nodigest_DDA_120min_...,DPN(+.98)LPLK(+42.01)H,8,98,8,488.7589,2,87.33,12100000.0,975.5025,0.7,Deamidation(N); Acetylation (K),93 99 100 100 98 99 99 97,DPN(+.98)LPLK(+42.01)H,HCD
4,22,8756,20210115_Weissrot_332_T12_nodigest_DDA_120min_...,KDETTLVDGH,10,97,10,557.7729,2,31.10,1210000.0,1113.5303,0.9,NaN,98 99 100 99 98 100 98 99 96 92,KDETTLVDGH,HCD


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine, lysine, proline, arginine, tyrosine: 15.9949 MKPRY
    variable deamidation of asparagine, glumatine: 0.984016 NQ
    variable methylation of lysine and arginine: 14.015650 KR
    variable pyro-glu from glutamine: -17.03 Q
    variable acetylation of lysine: 42.01 K


We'll then write this manipulated dataframe to a new file.

In [ ]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks332['A'] = peaks332['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks332['C'] = peaks332['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks332['D'] = peaks332['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks332['E'] = peaks332['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks332['F'] = peaks332['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks332['G'] = peaks332['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks332['H'] = peaks332['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks332 output, there will be no isoleucines (they're lumped in with leucines)
peaks332['I'] = peaks332['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks332['K'] = peaks332['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks332['L'] = peaks332['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks332['M'] = peaks332['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks332['N'] = peaks332['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks332['P'] = peaks332['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks332['Q'] = peaks332['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks332['R'] = peaks332['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks332['S'] = peaks332['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks332['T'] = peaks332['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks332['V'] = peaks332['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks332['W'] = peaks332['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks332['Y'] = peaks332['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks332['c-carb'] = peaks332['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks332['m-oxid'] = peaks332['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a count function to enumerate the # of oxidized K's in each peptide
peaks332['k-oxid'] = peaks332['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of oxidized P's in each peptide
peaks332['p-oxid'] = peaks332['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of oxidized R's in each peptide
peaks332['r-oxid'] = peaks332['Peptide'].apply(lambda x: x.count('R(+15.99)'))

# use a count function to enumerate the # of oxidized Y's in each peptide
peaks332['y-oxid'] = peaks332['Peptide'].apply(lambda x: x.count('Y(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks332['n-deam'] = peaks332['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
#peaks332['q-deam'] = peaks332['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of methylated K's in each peptide
peaks332['k-meth'] = peaks332['Peptide'].apply(lambda x: x.count('K(+14.02)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks332['r-meth'] = peaks332['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# use a count function to enumerate the # of pyro glu Q's in each peptide
peaks332['q-pyro'] = peaks332['Peptide'].apply(lambda x: x.count('Q(-17.03)'))

# use a count function to enumerate the # of acetylation of K's in each peptide
peaks332['k-acet'] = peaks332['Peptide'].apply(lambda x: x.count('K(+42.01)'))

# create a column with 'stripped' peptide sequences using strip
peaks332['stripped peptide'] = peaks332['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks332['stripped length'] = peaks332['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks332['ptm-total'] = peaks332['c-carb'] + peaks332['m-oxid'] + peaks332['k-oxid'] + peaks332['p-oxid'] \
+ peaks332['r-oxid'] + peaks332['y-oxid'] + peaks332['n-deam'] + peaks332['k-meth'] + peaks332['r-meth'] \
+ peaks332['q-pyro'] + peaks332['k-acet']

# calculate NAAF numerator for each peptide k
peaks332['NAAF num.'] = peaks332['Area'] / peaks332['stripped length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks332.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_332_T12_undigested_combine_PTMopt_DN80.csv")

# check out the results
peaks332.head()

In [ ]:
# made a new dataframe that contains the sums of certain columns 
# in the stripped peptide dataframe above (for >80% ALC)

index = ['sample total']

data = {'A': peaks332['A'].sum(),
        'C': peaks332['C'].sum(),
        'D': peaks332['D'].sum(),
        'E': peaks332['E'].sum(),
        'F': peaks332['F'].sum(),
        'G': peaks332['G'].sum(),
        'H': peaks332['H'].sum(),
        'I': peaks332['I'].sum(),
        'K': peaks332['K'].sum(),
        'L': peaks332['L'].sum(),
        'M': peaks332['M'].sum(),
        'N': peaks332['N'].sum(),
        'P': peaks332['P'].sum(),
        'Q': peaks332['Q'].sum(),
        'R': peaks332['R'].sum(),
        'S': peaks332['S'].sum(),
        'T': peaks332['T'].sum(),
        'V': peaks332['V'].sum(),
        'W': peaks332['W'].sum(),
        'Y': peaks332['Y'].sum(),
        'c-carb': peaks332['c-carb'].sum(),
        'm-oxid': peaks332['m-oxid'].sum(),
        'k-oxid': peaks332['k-oxid'].sum(),
        'p-oxid': peaks332['p-oxid'].sum(),
        'r-oxid': peaks332['r-oxid'].sum(),
        'y-oxid': peaks332['y-oxid'].sum(),
        'n-deam': peaks332['n-deam'].sum(),
        'k-meth': peaks332['k-meth'].sum(),
        'r-meth': peaks332['r-meth'].sum(),
        'q-pyro': peaks332['q-pyro'].sum(),
        'k-acet': peaks332['k-acet'].sum(),
        'Total area': peaks332['Area'].sum(),
        'Total length': peaks332['stripped length'].sum()
       }

totalpeaks332 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', \
                                            'y-oxid', 'n-deam', 'k-meth', 'r-meth', 'q-pyro', \
                                            'k-acet', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks332['% C w/ carb'] = totalpeaks332['c-carb'] / totalpeaks332['C'] 

# calculate percentage of M's that are oxidized
totalpeaks332['% M w/ oxid'] = totalpeaks332['m-oxid'] / totalpeaks332['M'] 

# calculate percentage of K's that are oxidized
totalpeaks332['% K w/ oxid'] = totalpeaks332['k-oxid'] / totalpeaks332['K'] 

# calculate percentage of P's that are oxidized
totalpeaks332['% P w/ oxid'] = totalpeaks332['p-oxid'] / totalpeaks332['P'] 

# calculate percentage of R's that are oxidized
totalpeaks332['% R w/ oxid'] = totalpeaks332['p-oxid'] / totalpeaks332['R'] 

# calculate percentage of Y's that are oxidized
totalpeaks332['% Y w/ oxid'] = totalpeaks332['y-oxid'] / totalpeaks332['Y'] 

# calculate percentage of N's that are deamidated
totalpeaks332['% N w/ deam'] = totalpeaks332['n-deam'] / totalpeaks332['N'] 

# calculate percentage of K's that are methylated
totalpeaks332['% K w/ meth'] = totalpeaks332['k-meth'] / totalpeaks332['K'] 

# calculate percentage of R's that are methylated
totalpeaks332['% R w/ meth'] = totalpeaks332['r-meth'] / totalpeaks332['R'] 

# calculate percentage of Q's that are pyro glu'd
totalpeaks332['% Q w/ pyro'] = totalpeaks332['q-pyro'] / totalpeaks332['Q'] 

# calculate percentage of K's that are acetylation
totalpeaks332['% K w/ acet'] = totalpeaks332['k-acet'] / totalpeaks332['K'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks332['NAAF denom.'] = totalpeaks332['Total area'] / totalpeaks332['Total length']

# write modified dataframe to new txt file
totalpeaks332.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_332_T12_undigested_combine_PTMopt_DN80_totals.csv")

totalpeaks332.head()

In [ ]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF80 = 243531.178471

# use NAAF >80% ALC to get NAAF factor
peaks332['NAAF factor'] = (peaks332['NAAF num.'])/NAAF80

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksAAPTM_332 = peaks332[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'k-oxid', 'p-oxid', 'r-oxid', 'y-oxid', 'n-deam', 'k-meth', 'r-meth', \
                                'q-pyro', 'k-acet']].copy()

# multiply the NAAF80 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_332['A-NAAF80'] = peaksAAPTM_332['A'] * peaks332['NAAF factor']
peaksAAPTM_332['C-NAAF80'] = peaksAAPTM_332['C'] * peaks332['NAAF factor']
peaksAAPTM_332['D-NAAF80'] = peaksAAPTM_332['D'] * peaks332['NAAF factor']
peaksAAPTM_332['E-NAAF80'] = peaksAAPTM_332['E'] * peaks332['NAAF factor']
peaksAAPTM_332['F-NAAF80'] = peaksAAPTM_332['F'] * peaks332['NAAF factor']
peaksAAPTM_332['G-NAAF80'] = peaksAAPTM_332['G'] * peaks332['NAAF factor']
peaksAAPTM_332['H-NAAF80'] = peaksAAPTM_332['H'] * peaks332['NAAF factor']
peaksAAPTM_332['I-NAAF80'] = peaksAAPTM_332['I'] * peaks332['NAAF factor']
peaksAAPTM_332['K-NAAF80'] = peaksAAPTM_332['K'] * peaks332['NAAF factor']
peaksAAPTM_332['L-NAAF80'] = peaksAAPTM_332['L'] * peaks332['NAAF factor']
peaksAAPTM_332['M-NAAF80'] = peaksAAPTM_332['M'] * peaks332['NAAF factor']
peaksAAPTM_332['N-NAAF80'] = peaksAAPTM_332['N'] * peaks332['NAAF factor']
peaksAAPTM_332['P-NAAF80'] = peaksAAPTM_332['P'] * peaks332['NAAF factor']
peaksAAPTM_332['Q-NAAF80'] = peaksAAPTM_332['Q'] * peaks332['NAAF factor']
peaksAAPTM_332['R-NAAF80'] = peaksAAPTM_332['R'] * peaks332['NAAF factor']
peaksAAPTM_332['S-NAAF80'] = peaksAAPTM_332['S'] * peaks332['NAAF factor']
peaksAAPTM_332['T-NAAF80'] = peaksAAPTM_332['T'] * peaks332['NAAF factor']
peaksAAPTM_332['V-NAAF80'] = peaksAAPTM_332['V'] * peaks332['NAAF factor']
peaksAAPTM_332['W-NAAF80'] = peaksAAPTM_332['W'] * peaks332['NAAF factor']
peaksAAPTM_332['Y-NAAF80'] = peaksAAPTM_332['Y'] * peaks332['NAAF factor']

# multiply the NAAF80 factor by the PTMs normalize its abundance by peak area and peptide length

peaksAAPTM_332['ccarb-NAAF80'] = peaksAAPTM_332['c-carb'] * peaksAAPTM_332['NAAF factor']
peaksAAPTM_332['moxid-NAAF80'] = peaksAAPTM_332['m-oxid'] * peaksAAPTM_332['NAAF factor']
peaksAAPTM_332['koxid-NAAF80'] = peaksAAPTM_332['k-oxid'] * peaksAAPTM_332['NAAF factor']
peaksAAPTM_332['poxid-NAAF80'] = peaksAAPTM_332['p-oxid'] * peaksAAPTM_332['NAAF factor']
peaksAAPTM_332['roxid-NAAF80'] = peaksAAPTM_332['r-oxid'] * peaksAAPTM_332['NAAF factor']
peaksAAPTM_332['yoxid-NAAF80'] = peaksAAPTM_332['y-oxid'] * peaksAAPTM_332['NAAF factor']
peaksAAPTM_332['ndeam-NAAF80'] = peaksAAPTM_332['n-deam'] * peaksAAPTM_332['NAAF factor']
peaksAAPTM_332['kmeth-NAAF80'] = peaksAAPTM_332['k-meth'] * peaksAAPTM_332['NAAF factor']
peaksAAPTM_332['rmeth-NAAF80'] = peaksAAPTM_332['r-meth'] * peaksAAPTM_332['NAAF factor']
peaksAAPTM_332['qpyro-NAAF80'] = peaksAAPTM_332['q-pyro'] * peaksAAPTM_332['NAAF factor']
peaksAAPTM_332['kacet-NAAF80'] = peaksAAPTM_332['k-acet'] * peaksAAPTM_332['NAAF factor']

# write the dataframe to a new csv
peaksAAPTM_332.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_332_T12_undigested_combine_PTMopt_DN80_NAAF.csv")

peaksAAPTM_332.head()

In [ ]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMs

index = ['sample total']

data = {'NAAF': peaksAAPTM_332['NAAF factor'].sum(),
        'A': peaksAAPTM_332['A-NAAF80'].sum(),
        'C': peaksAAPTM_332['C-NAAF80'].sum(),
        'D': peaksAAPTM_332['D-NAAF80'].sum(),
        'E': peaksAAPTM_332['E-NAAF80'].sum(),
        'F': peaksAAPTM_332['F-NAAF80'].sum(),
        'G': peaksAAPTM_332['G-NAAF80'].sum(),
        'H': peaksAAPTM_332['H-NAAF80'].sum(),
        'I': peaksAAPTM_332['I-NAAF80'].sum(),
        'K': peaksAAPTM_332['K-NAAF80'].sum(),
        'L': peaksAAPTM_332['L-NAAF80'].sum(),
        'M': peaksAAPTM_332['M-NAAF80'].sum(),
        'N': peaksAAPTM_332['N-NAAF80'].sum(),
        'P': peaksAAPTM_332['P-NAAF80'].sum(),
        'Q': peaksAAPTM_332['Q-NAAF80'].sum(),
        'R': peaksAAPTM_332['R-NAAF80'].sum(),
        'S': peaksAAPTM_332['S-NAAF80'].sum(),
        'T': peaksAAPTM_332['T-NAAF80'].sum(),
        'V': peaksAAPTM_332['V-NAAF80'].sum(),
        'W': peaksAAPTM_332['W-NAAF80'].sum(),
        'Y': peaksAAPTM_332['Y-NAAF80'].sum(),
        'c-carb': peaksAAPTM_332['ccarb-NAAF80'].sum(),
        'm-oxid': peaksAAPTM_332['moxid-NAAF80'].sum(),
        'k-oxid': peaksAAPTM_332['koxid-NAAF80'].sum(),
        'p-oxid': peaksAAPTM_332['poxid-NAAF80'].sum(),
        'r-oxid': peaksAAPTM_332['roxid-NAAF80'].sum(),
        'y-oxid': peaksAAPTM_332['yoxid-NAAF80'].sum(),
        'n-deam': peaksAAPTM_332['ndeam-NAAF80'].sum(),
        'k-meth': peaksAAPTM_332['kmeth-NAAF80'].sum(),
        'r-meth': peaksAAPTM_332['rmeth-NAAF80'].sum(),
        'q-pyro': peaksAAPTM_332['qpyro-NAAF80'].sum(),
        'k-acet': peaksAAPTM_332['kacet-NAAF80'].sum()
       }

totalpeaks80_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', \
                                           'r-oxid', 'y-oxid', 'n-deam', 'k-meth', 'r-meth', \
                                            'q-pyro', 'k-acet'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks80_NAAF['% C w/ carb.'] = totalpeaks80_NAAF['c-carb'] / totalpeaks80_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks80_NAAF['% M w/ oxid'] = totalpeaks80_NAAF['m-oxid'] / totalpeaks80_NAAF['M'] 

# calculate NAAF-corrected percentage of K's that are oxidized
totalpeaks80_NAAF['% K w/ oxid'] = totalpeaks80_NAAF['k-oxid'] / totalpeaks80_NAAF['K'] 

# calculate NAAF-corrected percentage of P's that are oxidized
totalpeaks80_NAAF['% P w/ oxid'] = totalpeaks80_NAAF['p-oxid'] / totalpeaks80_NAAF['P'] 

# calculate NAAF-corrected percentage of R's that are oxidized
totalpeaks80_NAAF['% R w/ oxid'] = totalpeaks80_NAAF['r-oxid'] / totalpeaks80_NAAF['R'] 

# calculate NAAF-corrected percentage of Y's that are oxidized
totalpeaks80_NAAF['% Y w/ oxid'] = totalpeaks80_NAAF['y-oxid'] / totalpeaks80_NAAF['Y'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks80_NAAF['% N w/ deam'] = totalpeaks80_NAAF['n-deam'] / totalpeaks80_NAAF['N'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaks80_NAAF['% K w/ meth'] = totalpeaks80_NAAF['k-meth'] / totalpeaks80_NAAF['K'] 

# calculate NAAF-corrected percentage of R's that are methylated
totalpeaks80_NAAF['% R w/ meth'] = totalpeaks80_NAAF['r-meth'] / totalpeaks80_NAAF['R'] 

# calculate NAAF-corrected percentage of Q's that are pyro glu'd
totalpeaks80_NAAF['% Q w/ pyro'] = totalpeaks80_NAAF['q-pyro'] / totalpeaks80_NAAF['Q'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaks80_NAAF['% K w/ acet'] = totalpeaks80_NAAF['k-acet'] / totalpeaks80_NAAF['K'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks80_NAAF['NAAF check'] = totalpeaks80_NAAF['NAAF'] / 243531.178471

# write modified dataframe to new txt file, same name + totals
totalpeaks80_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_332_T12_undigested_combine_PTMopt_DN80_NAAF_totals.csv")

totalpeaks80_NAAF.head()

## Export stripped peptides >80% ALC

In [ ]:
# keep only stripped peptide column 
pep80 = peaks332[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep80.to_csv("/home/millieginty/Documents/git-repos/TW_332_T12_undigested_combine_PTMopt_DN80_stripped_peptides.txt", header=False, index=False)

# removing redundancy
peaks80dedup = pd.DataFrame.drop_duplicates(pep80)

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

peaks80dedup.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_332_T12_undigested_combine_PTMopt_DN80_nonredundant_stripped_peptides.txt", header=False, index=False)

print("# redundant stripped Peaks peptides >80% ALC", len(pep80))
print("# nonredundant stripped Peaks peptides >80% ALC", len(peaks80dedup))
print("average peptide length Peaks peptides >80% ALC", peaks332['stripped length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >80% ALC
pep80m = peaks332[["Peptide"]]

# deduplicate
pep80mdedup = pd.DataFrame.drop_duplicates(pep80m)

print("# redundant Peaks peptides >80% ALC", len(pep80m))
print("# nonredundant Peaks peptides", len(pep80mdedup))

# check
pep80.head()